In [ ]:
import os
import sys
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] + "/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] + "/pyspark.zip")


## Create SparkContext, SparkSession
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext()

In [ ]:
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

In [ ]:
data = sc.textFile("/user/HistoricalData/")

In [ ]:
data.count()

3296451

In [ ]:
def convertDataFloat(line):
    return array([float(line[1]),float(line[2])])

In [ ]:
fea_data = data.map(lambda data:data.split(','))
parsedData = fea_data.map(lambda line : convertDataFloat(line))

In [ ]:
parsedData.take(5)

[array([ 40.7586, -73.9706]),
 array([ 40.7605, -73.9994]),
 array([ 40.732 , -73.9999]),
 array([ 40.7635, -73.9793]),
 array([ 40.7204, -74.0047])]

In [ ]:
clusters = KMeans.train(parsedData,8, maxIterations=10,runs=10, initializationMode="random")

/usr/hdp/current/spark2-client/python/lib/pyspark.zip/pyspark/mllib/clustering.py:347: UserWarning: The param `runs` has no effect since Spark 2.0.0.


In [ ]:
clusters.centers

[array([ 40.66634105, -73.98116308]),
 array([ 40.91692663, -73.85494899]),
 array([ 40.71025606, -73.94658291]),
 array([ 40.78451207, -73.95887343]),
 array([ 40.76251203, -73.86883806]),
 array([ 40.6645525 , -73.75705697]),
 array([ 40.72328514, -74.01019258]),
 array([ 40.75169967, -73.98404182])]

In [ ]:
def wsssError(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

In [ ]:
WSSSE = parsedData.map(lambda point: wsssError(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

Within Set Sum of Squared Error = 68813.0026937


In [ ]:
#clusters.predict(array([40.7204,-74.0047]))

6

In [ ]:
#sqrt(sum((array([40.7204,-74.0047]) - array([ 40.71743048, -74.002436  ])) ** 2))

0.0037341324334293183

In [ ]:
#40.7204 - 40.71743048

0.002969520000000614

In [ ]:
#-74.0047 - -74.002436

-0.002263999999996713

In [ ]:
clusters.save(sc, "/user/Data/Model/kmeanModel")